## 예제 11-1 : 함수 체인 


In [3]:
def func1(x) :
    print("fun1 call")
    return f[x]

def func2(x) :
    print("fun2 call")
    return f[x]

def func3() :
    print("fun3 call")
    return " exit "

In [4]:
f = {1 :func1, 2: func2, "exit":func3}

In [5]:
print(func1(2)("exit")())

fun1 call
fun2 call
fun3 call
 exit 


In [5]:
a = (lambda x : (lambda x : (lambda x : x+1)(x))(x))
print(a(5))

6


In [11]:
a = lambda x : func1(x)

print(a(2)("exit")())
    

fun1 call
fun2 call
fun3 call
 exit 


## 예제 11-2 : 클로저 환경 및 속성 확인하기 


In [6]:
def outer(x) :
    def inner(y) :
        return x+y
    return inner

In [7]:
inner = outer(5)

print(inner)
print(inner.__name__)
print(inner.__closure__)
print(inner.__code__.co_freevars)

<function outer.<locals>.inner at 0x0000000005180EA0>
inner
(<cell at 0x0000000005131E58: int object at 0x00000000626BBA50>,)
('x',)


In [8]:
print(inner.__closure__[0])
print(type(inner.__closure__[0]))

<cell at 0x0000000005131E58: int object at 0x00000000626BBA50>
<class 'cell'>


In [9]:
for i in dir(inner.__closure__[0]) :
    if not i.startswith("__") :
        print(i)

cell_contents


In [24]:
print(inner.__closure__[0].cell_contents)

5


In [25]:
print(inner(10))

15


In [10]:
cf = lambda x : (lambda y : x+y )


In [11]:
b = cf(5)

print(b.__closure__)
print(b.__closure__[0].cell_contents)

(<cell at 0x0000000005131B28: int object at 0x00000000626BBA50>,)
5


In [12]:
b(10)

15

## 예제 11-3  :  클로저를 이용한 커링(currying)처리

In [33]:
import operator as op

op = {"//":op.floordiv,"%":op.mod}

def outer(x) :
    def inner(y) :
        return op["//"](x,y), op["%"](x,y)
    return inner


In [34]:
inner = outer(123)
print(inner(5))

(24, 3)


In [36]:
def outer(op) :
    def inner(x,y) :
         return op["//"](x,y), op["%"](x,y)
    return inner

In [37]:
inner = outer(op)
print(inner(123,5))

(24, 3)


## 예제 11-4 :  내장 모듈을 이용한 커링 


In [14]:
import functools as ft

print(ft.partial)

<class 'functools.partial'>


In [15]:
import operator as op

s = ft.partial(op.add,5)

print(s)

functools.partial(<built-in function add>, 5)


In [16]:
for i in dir(s) :
    if not i.startswith("__") :
        print(i)

args
func
keywords


In [17]:
print(s.args)
print(s.func)
print(s.keywords)

(5,)
<built-in function add>
{}


In [18]:
print(s(10))
print(s.func(s.args[0],10))

15
15


In [19]:
import functools as ft 

def addx(x,y,z) :
    return x+y+z

s = ft.partial(addx,1,z=100)

print(s.args)
print(s.keywords)

(1,)
{'z': 100}


In [20]:
print(s(20))


121


In [22]:
print(s(x=10))

TypeError: addx() got multiple values for argument 'x'

## 예제 11-5 :  가변인자를 활용한 커링


In [23]:
import functools as ft 

def addx(x,y,*,z,**kwargs) :
    result = x+y+z
    for v in kwargs.values() :
        result = result + v
    return result


In [24]:
s = ft.partial(addx,1,1,z=10)

In [25]:
print(s(k=10))
print(s.args, s.keywords)

print(s(a =100))
print(s.args, s.keywords)

22
(1, 1) {'z': 10}
112
(1, 1) {'z': 10}


In [67]:
import functools as ft 

ss = ft.partial(sum)

In [66]:
print(ss([1,2,3]))
print(ss.args, ss.keywords)
print(ss([1,2,3,4,5,6]))
print(ss.args, ss.keywords)

6
() {}
21
() {}


## 예제 11-6 :  functools.partialmethod를 이용한 메소드 커링처리


In [68]:
import functools as ft 

class A :
    @ft.partialmethod
    def add(self,x,y) :
        return x+y
    

In [72]:
a = A()

print(a.add)
print(a.add.args)
print(a.add.func)
print(a.add.keywords)

functools.partial(<bound method A.add of <__main__.A object at 0x00000000050005F8>>)
()
<bound method A.add of <__main__.A object at 0x00000000050005F8>>
{}


In [76]:
print(a.add(5,5))

10


In [98]:
import functools as ft 

class A :
    @ft.partialmethod
    def add(self,x) :
        def inner(y) :
            return x+y
        return inner


In [106]:
a = A()
print(a)

print(a.add)
print(a.add.args)

functools.partial(<bound method A.add of <__main__.A object at 0x000000000503B278>>)
()


In [108]:
add = a.add(5)
print(add)
print(add(5))

<function A.add.<locals>.inner at 0x00000000050336A8>
10


## 예제 11-7 :  사용자 클래스로 부분함수 만들기

In [31]:
class part :
    def __init__(self,func,*args) :
        self.func = func
        self.args = []
        if len(args) :
            for i in args :
                self.args.append(i)       
        
    def __call__(self,*args) : 
        if len(args) :
            for i in args :
                self.args.append(i)
        if self.func.__code__.co_argcount == 1 :
            return self.func(self.args)
        else :           
            return self.func(*self.args)
        

In [32]:
def add(x,y) :
    return x+y

a = part(add,5)
print(a.args)

[5]


In [33]:
print(a(5))

10


In [34]:
def list_sum(l) :
    return sum(l)

s = part(list_sum)
print(s.args)

[]


In [35]:
print(s(1,2,3,4,5))

15


## 예제 11-8  : 함수의 객체 영역 이용하기

In [16]:
def fib(n) :
    if (n == 0) or (n== 1) :
        return 1
    return fib(n-1) + fib(n-2)


In [17]:
a = fib(5)

In [18]:
print(a)

8


In [19]:
def memoize(func) :
    memoize.cache = {} 
    def g(x) :
        if x not in memoize.cache :
            memoize.cache[x] = func(x)
        return memoize.cache[x]
    return g


In [20]:
a = memoize(fib)
s = a(5)

In [21]:
memoize.cache

{5: 8}

In [22]:
fib = memoize(fib)

for i in range(15) :
    print(fib(i),end= ' ')
    

1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 

In [23]:
import pprint

pprint.pprint(memoize.cache)

{0: 1,
 1: 1,
 2: 2,
 3: 3,
 4: 5,
 5: 8,
 6: 13,
 7: 21,
 8: 34,
 9: 55,
 10: 89,
 11: 144,
 12: 233,
 13: 377,
 14: 610}


## 예제 11-9  : 메모이제이션을 functools.lru_cache 사용


In [36]:
from functools import lru_cache

print(lru_cache)

<function lru_cache at 0x0000000002571158>


In [37]:
print(lru_cache.__code__.co_varnames)

('maxsize', 'typed', 'decorating_function')


In [38]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n) :
    if n < 2 :
        return 1
    return fib(n-1) + fib(n-2)

In [42]:
print(fib)

In [41]:
for i in dir(fib) :
    if not i.startswith("_") :
        print(i)

cache_clear
cache_info


In [31]:
print([fib(n) for n in range(15)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]


In [33]:
print(fib.cache_info())
fib.cache_clear()
print(fib.cache_info())

CacheInfo(hits=26, misses=15, maxsize=None, currsize=15)
CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)


In [34]:
print([fib(n) for n in range(15)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]


In [35]:
print(fib.cache_info())

CacheInfo(hits=26, misses=15, maxsize=None, currsize=15)


## 예제 11-10  :  기본 데코레이터 처리  


In [40]:
def decorator(func) :
    return func

@decorator
def add(x,y) :
    return x+y

print(add(5,5))

10


In [43]:
def decorator(func) :
    return func

def add(x,y) :
    return x+y

add = decorator(add)
print(add(5,5))

10


In [16]:
lambda_deco = lambda func : func
    
@lambda_deco
def add(x,y) :
    return x+y

In [17]:
print(add.__name__)
print(add(5,5))

add
10


## 예제 11-11  : 데코레이터 처리하는 내부함수 정의하기


In [43]:
def decorator(func) :
    def wrapper(*args,**kwargs) :
        """ wrapper call """
        return func(*args,**kwargs)
    return wrapper


In [44]:
@decorator
def add(x,y) :
    """ add call """
    return x+y

print(add(5,51))

56


In [61]:
for i in add.__dict__ :
    print(i)

In [45]:
print(add.__name__)
print(add.__qualname__)
print(add.__doc__)


wrapper
decorator.<locals>.wrapper
 wrapper call 


In [46]:
lambda_deco = lambda func : (lambda *args : func(*args))
    
@lambda_deco
def add(x,y) :
    return x+y

In [47]:
print(add.__name__)
print(add(5,5))


<lambda>
10


## 예제 11-12  :  실행함수의 함수 정보를 유지하기


In [48]:
from functools import wraps

def decorator(func) :
    
    @wraps(func)
    def wrapper(*args,**kwargs) :
        """ wrapper call """
        return func(*args,**kwargs)
    return wrapper


In [49]:
@decorator
def add(x,y) :
    """ add call """
    return x+y

print(add(5,51))

56


In [50]:
for i in add.__dict__ :
    print(i)

__wrapped__


In [51]:
print(add.__name__)
print(add.__qualname__)
print(add.__doc__)
print(add.__wrapped__)

add
add
 add call 
<function add at 0x00000000052B0A60>


In [52]:
print(add.__wrapped__.__name__)

add


In [53]:
print(add.__wrapped__(5,51))

56


## 예제 11-13  : 공통 함수를 내부 함수에 추가 


In [6]:
import collections.abc as cols


def typecheck(args,kwargs) :
    print("{:-^40s}".format(" wrapper "))
    if isinstance(args, cols.Sequence) :
        print(" args type ", "tuple")
        
    if isinstance(kwargs, cols.Mapping) :
        print(" kwargs type ", "dict")
        
    print("{:-^40s}".format(""))

In [7]:
from functools import wraps

def dec_func(func) :
    @wraps(func)
    def wrapper1(*args, **kwargs) :
        typecheck(args,kwargs)
        return func(*args, **kwargs)  
    return wrapper1

In [8]:
@dec_func
def add(x,y) :
    "doc add "
    return x+y

print(add.__name__)
print(add.__doc__)
print(add(5,5))

add
doc add 
--------------- wrapper ----------------
 args type  tuple
 kwargs type  dict
----------------------------------------
10


## 예제 11-14  :   테코레이터 함수에 매개변수 정의하기


In [76]:
from functools import wraps

def out_para(x) :
    print(x)
    def dec_func(func) :
        @wraps(func)
        def wrapper(*args, **kwargs) :
            return func(*args, **kwargs)
        
        return wrapper
    return dec_func


In [77]:
@out_para("decorator parameter")
def add(x,y) :
    return x+y

print(add(5,5))
print(add.__name__)

decorator parameter
10
add


In [79]:
dec_func = out_para(" first func ")
wrapper = dec_func(add)
print(wrapper(5,5))

 first func 
10


## 예제 11-15  : 클래스에 함수 데코레이터 처리하기 

In [55]:
import pprint

def decorator(cls) :
    cls.a = "insert attribute"
    return cls


In [56]:
@decorator
class A :
    pass

print(A.a)


insert attribute


In [57]:
ai = A()
pprint.pprint(A.__dict__)

mappingproxy({'__dict__': <attribute '__dict__' of 'A' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              'a': 'insert attribute'})


## 예제 11-16  : 클래스로 테코레이터 정의 

In [58]:
class Prop_dec(object):
    "Emulate PyProperty_Type() in Objects/descrobject.c"
    
    def __init__(self, fget=None):
        self.fget = fget
        Prop_dec.add = self
    
    def __call__(self, obj, objtype=None):
        return self.fget(obj)

In [59]:
import pprint

pprint.pprint(Prop_dec.__dict__)

mappingproxy({'__call__': <function Prop_dec.__call__ at 0x00000000052D4400>,
              '__dict__': <attribute '__dict__' of 'Prop_dec' objects>,
              '__doc__': 'Emulate PyProperty_Type() in Objects/descrobject.c',
              '__init__': <function Prop_dec.__init__ at 0x00000000052D4F28>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Prop_dec' objects>})


In [60]:
@Prop_dec
def add(x):
    return x['x'], x['y']

In [61]:
print(add)
print(add.__dict__)
print(add({'x':5, 'y':5}))

{'fget': <function add at 0x00000000052B0AE8>}
(5, 5)


In [62]:
add = Prop_dec(add)
print(add.__dict__)
print(add({'x':5, 'y':5}))

{'fget': <__main__.Prop_dec object at 0x00000000052BF588>}
(5, 5)


## 예제 11-17  :  클래스 메소드를 데코레이터 

In [42]:
class DECC :
    @classmethod
    def attr_check(cls, Base) :
        print(" DECC ")
        Base.a = "decc"
        return Base

In [43]:
@DECC.attr_check
def add(x,y) :
    return x+y

 DECC 


In [44]:
print(add)
print(add(5,5))
print(add.a)

<function add at 0x0000000004C3AEA0>
10
decc


## 예제 11-18  : 인스턴스 메소드로 함수에 대한 데코레이터 처리


In [45]:
class DECD :
            
    def attr_check(self, Base) :
        print(" DECD ")
        Base.a = "decd"
        return Base


In [46]:
d = DECD()
@d.attr_check
def add(x,y) :
    return x+y

 DECD 


In [47]:
print(add)
print(add(5,5))
print(add.a)

<function add at 0x0000000004C3AE18>
10
decd


## 예제 11-19  : 데코레이터를 이용해서 type체크를 위한 작업


In [50]:
def interface(*attributes):
    def decorator(Base):

        def checker(Other):
            return all(hasattr(Other, a) for a in attributes)

        def __subclasshook__(cls, Other):
            if checker(Other):
                return True
            return NotImplemented

        def __instancecheck__(cls, Other):
            return checker(Other)

        Base.__subclasshook__ = classmethod(__subclasshook__)
        Base.__instancecheck__ = classmethod(__instancecheck__)
        return Base

    return decorator

In [63]:
from abc import ABC
@interface("x", "y")
class Foo(ABC):
    
    def x(self): return 5
    def y(self): return 10


In [64]:
class Bar(object):
    def x(self): return "blah"
    def y(self): return "blah"

class Baz(object):
    def __init__(self):
        self.x = "blah"
        self.y = "blah"

class attrdict(dict):
    def __getattr__(self, attr):
        return self[attr]
    

In [67]:
b = Bar()
z = Baz()
t = attrdict({"x":27.5, "y":37.5})

print(isinstance(b, Foo))
print(issubclass(Bar, Foo))
print(isinstance(z, Foo))
print(isinstance(t, Foo))

True
True
False
False
